![Factories creating emissions](pollution.jpg)

Photo by Maxim Tolchinskiy on Unsplash
  
When factoring heat generation required for the manufacturing and transportation of products, _Greenhouse gas emissions attributable to products, from food to sneakers to appliances, make up more than 75% of global emissions._ (`Source: The Carbon Catalogue https://www.nature.com/articles/s41597-022-01178-9`)

Our data, which is publicly available on nature.com, contains product carbon footprints (PCFs) for various companies. PCFs are the greenhouse gas emissions attributable to a given product, measured in CO<sub>2</sub> (carbon dioxide equivalent).
<!--https://www.nature.com/articles/s41597-022-01178-9-->

This data is stored in a PostgreSQL database containing one table, `product_emissions`, which looks at PCFs by product as well as the stage of production that these emissions occurred. Here's a snapshot of what `product_emissions` contains in each column:

### `product_emissions`

| field                              | data type |
|------------------------------------|-----------|
| `id`                                 | `VARCHAR`   |
| `year`                               | `INT`       |
| `product_name`                       | `VARCHAR`   |
| `company`                            | `VARCHAR`   |
| `country`                            | `VARCHAR`   |
| `industry_group`                     | `VARCHAR`   |
| `weight_kg`                          | `NUMERIC`   |
| `carbon_footprint_pcf`               | `NUMERIC`   |
| `upstream_percent_total_pcf`         | `VARCHAR`   |
| `operations_percent_total_pcf`       | `VARCHAR`   |
| `downstream_percent_total_pcf`       | `VARCHAR`   |

You'll use this data to examine the carbon footprint of each industry in the dataset! 

This initial query provides a quick glimpse into the emissions_data table.

In [5]:
SELECT *
FROM public.product_emissions
LIMIT 10;

,id,year,product_name,company,country,industry_group,weight_kg,carbon_footprint_pcf,upstream_percent_total_pcf,operations_percent_total_pcf,downstream_percent_total_pcf
0,10056-1-2014,2014,Frosted Flakes(R) Cereal,Kellogg Company,USA,"Food, Beverage & Tobacco",0.7485,2.00,57.50%,30.00%,12.50%
1,10056-1-2015,2015,"Frosted Flakes, 23 oz, produced in Lancaster, ...",Kellogg Company,USA,Food & Beverage Processing,0.7485,2.00,57.50%,30.00%,12.50%
2,10222-1-2013,2013,Office Chair,KNOLL INC,USA,Capital Goods,20.6800,72.54,80.63%,17.36%,2.01%
3,10261-1-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.0000,1488.00,30.65%,5.51%,63.84%
4,10261-2-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.0000,1818.00,25.08%,4.51%,70.41%
5,10261-3-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.0000,2274.00,20.05%,3.61%,76.34%
6,10324-1-2016,2016,KURALON fiber,"Kuraray Co., Ltd.",Japan,Materials,1500.0000,10000.00,N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data)
7,10418-1-2013,2013,Portland Cement,Lafarge S.A.,France,Materials,1000.0000,1102.00,N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data)
8,10661-10-2014,2014,Regular Straight 505® Jeans – Steel (Water<Less™),Levi Strauss & Co.,USA,Consumer Durables & Apparel,0.7665,15.00,N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data)
9,10661-10-2015,2015,Regular Straight 505® Jeans – Steel (Water<Less™),Levi Strauss & Co.,USA,"Textiles, Apparel, Footwear and Luxury Goods",0.7665,15.00,N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data)


# Getting familiar with the data
Query 2 gives the total number of records, indicating the size of the dataset we're working with.
Query 3 lists all the unique industries represented. This helps in understanding the categories we'll be analyzing and can reveal potential inconsistencies in naming conventions. Ordering the results alphabetically (ORDER BY industry_name) makes it easier to scan.
Query 4 identifies the earliest and latest years in the dataset, which is crucial for understanding the temporal coverage and for filtering to the "most recent year" as required by the project

In [6]:
SELECT COUNT(*) AS total_records
FROM product_emissions;

,total_records
0,866


In [8]:
SELECT DISTINCT industry_group
FROM public.product_emissions
ORDER BY industry_group;

,industry_group
0,Automobiles & Components
1,Capital Goods
2,Chemicals
3,Commercial & Professional Services
4,Consumer Durables & Apparel
5,"Consumer Durables, Household and Personal Prod..."
6,Containers & Packaging
7,Electrical Equipment and Machinery
8,Energy
9,Food & Beverage Processing


In [9]:
SELECT MIN(year) as earliest_year, MAX(year) as latest_year
FROM product_emissions;

,earliest_year,latest_year
0,2013,2017


# Data Cleaning
These two queries check for missing values and any inconsistences particularly regarding the industry and the pcfs

In [1]:
SELECT COUNT(*) AS records_with_missing_emissions
FROM product_emissions
WHERE carbon_footprint_pcf IS NULL;

,records_with_missing_emissions
0,0


In [3]:
SELECT LOWER(industry_group) AS industry_lowercase, COUNT(*) AS count
FROM product_emissions
GROUP BY LOWER(industry_group)
ORDER BY count DESC;

,industry_lowercase,count
0,technology hardware & equipment,195
1,materials,161
2,"food, beverage & tobacco",101
3,automobiles & components,73
4,consumer durables & apparel,52
5,commercial & professional services,42
6,capital goods,33
7,chemicals,29
8,software & services,27
9,food & staples retailing,24


# Exploratory Data Analysis
##  Carbon Footprint Analysis by Industry (Most Recent Year)

### Objective:
Explore the total carbon footprint of various industries in the **most recent year available** and identify the **highest-emitting sectors**. This query supports environmental impact analysis and industry-level emission comparisons.

In [8]:
SELECT MAX(year) as most_recent_yr
FROM product_emissions;

,most_recent_yr
0,2017


In [7]:
SELECT industry_group, SUM(carbon_footprint_pcf) as pcf_sum
FROM product_emissions
WHERE year=2017
GROUP BY industry_group
ORDER BY pcf_sum DESC;

,industry_group,pcf_sum
0,Materials,107129.0000
1,Capital Goods,94942.6724
2,Technology Hardware & Equipment,21865.0860
3,"Food, Beverage & Tobacco",3161.4700
4,Commercial & Professional Services,740.6000
5,Software & Services,690.0000
